In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [3]:
df = pd.read_csv('insurance_data.csv')
df

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [9]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(optimizer='adam',loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 578us/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 829us/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 571us/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 711us/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 555us/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 711us/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 416us/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 810us/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/100


1/1 [==============================] - 0s 503us/step - loss: 0.6835 - accuracy: 0.5000
Epoch 84/100
1/1 [==============================] - 0s 713us/step - loss: 0.6832 - accuracy: 0.5000
Epoch 85/100
1/1 [==============================] - 0s 885us/step - loss: 0.6829 - accuracy: 0.5000
Epoch 86/100
1/1 [==============================] - 0s 998us/step - loss: 0.6826 - accuracy: 0.5000
Epoch 87/100
1/1 [==============================] - 0s 760us/step - loss: 0.6823 - accuracy: 0.5000
Epoch 88/100
1/1 [==============================] - 0s 544us/step - loss: 0.6820 - accuracy: 0.5000
Epoch 89/100
1/1 [==============================] - 0s 506us/step - loss: 0.6817 - accuracy: 0.5000
Epoch 90/100
1/1 [==============================] - 0s 441us/step - loss: 0.6814 - accuracy: 0.5000
Epoch 91/100
1/1 [==============================] - 0s 729us/step - loss: 0.6811 - accuracy: 0.5000
Epoch 92/100
1/1 [==============================] - 0s 594us/step - loss: 0.6808 - accuracy: 0.5000
Epoch 93/100


In [10]:
model.predict(X_test_scaled)

array([[0.7743647 ],
       [0.72521067],
       [0.5345471 ],
       [0.7428877 ],
       [0.7775141 ],
       [0.7957479 ]], dtype=float32)

In [11]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [12]:
coef,intercept = model.get_weights()

In [13]:
coef,intercept

(array([[0.905744 ],
        [0.9045079]], dtype=float32),
 array([-0.09708446], dtype=float32))

In [14]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [15]:
X_test

,age,affordibility
2,47,1
10,18,1
21,26,0
11,28,1
14,49,1
9,61,1


In [19]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [20]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [25]:
def gradient_descent(age,affordability,y_true,epochs,loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.05
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age+w2*affordability+bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true,y_predicted)
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 
        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate*w1d
        w2 = w2 - rate*w2d
        bias = bias - bias_d * rate
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [26]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.4631)

Epoch:0, w1:0.9974907633470177, w2:0.9948348125394529, bias:-0.011341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9950375769407711, w2:0.9897573301834921, bias:-0.02254239435585742, loss:0.7081299930306214
Epoch:2, w1:0.9926400961624582, w2:0.9847672047756871, bias:-0.03360252203374183, loss:0.7050064771400743
Epoch:3, w1:0.9902979616201892, w2:0.9798640569784323, bias:-0.044523227390841844, loss:0.7019680236226099
Epoch:4, w1:0.988010799711406, w2:0.9750474771847867, bias:-0.055305520065079175, loss:0.6990128801936124
Epoch:5, w1:0.9857782231926034, w2:0.9703170264571072, bias:-0.06595044127049327, loss:0.6961392963774813
Epoch:6, w1:0.9835998317547898, w2:0.9656722374891652, bias:-0.07645906234744362, loss:0.6933455254515805
Epoch:7, w1:0.9814752126031583, w2:0.9611126155884899, bias:-0.08683248330761242, loss:0.6906298263156959
Epoch:8, w1:0.9794039410394907, w2:0.9566376396757436, bias:-0.09707183137729666, loss:0.6879904652845373
Epoch:9, w1:0.977385581045863, w2:0.95224676

Epoch:114, w1:0.9618073720487751, w2:0.8089973406279953, bias:-0.6772677703873144, loss:0.6032752607255548
Epoch:115, w1:0.9627369354466778, w2:0.8093588391712649, bias:-0.6799890955644288, loss:0.6031056033258184
Epoch:116, w1:0.9636762129660377, w2:0.8097348506644263, bias:-0.6826854312828334, loss:0.6029381330005109
Epoch:117, w1:0.9646250224487729, w2:0.8101250607664087, bias:-0.6853572218355725, loss:0.6027727748313949
Epoch:118, w1:0.9655831850160282, w2:0.8105291608031931, bias:-0.6880049034816834, loss:0.6026094566616873
Epoch:119, w1:0.9665505250161253, w2:0.8109468476818132, bias:-0.6906289045759844, loss:0.6024481089962282
Epoch:120, w1:0.9675268699729935, w2:0.8113778238049465, bias:-0.6932296456975542, loss:0.6022886649050788
Epoch:121, w1:0.9685120505350967, w2:0.8118217969861334, bias:-0.6958075397768775, loss:0.6021310599304379
Epoch:122, w1:0.9695059004248661, w2:0.8122784803656569, bias:-0.6983629922216267, loss:0.6019752319967799
Epoch:123, w1:0.9705082563886538, w2:

Epoch:230, w1:1.1050008301620464, w2:0.8972324773588704, bias:-0.8980040597932791, loss:0.5895589886630881
Epoch:231, w1:1.1063868983285798, w2:0.898131032413799, bias:-0.8994610872266972, loss:0.5894618287891442
Epoch:232, w1:1.107774008374012, w2:0.8990294034602108, bias:-0.9009142881172371, loss:0.5893648414553108
Epoch:233, w1:1.109162140261093, w2:0.8999275600072076, bias:-0.9023637119414629, loss:0.5892680253456539
Epoch:234, w1:1.1105512743308226, w2:0.9008254722280146, bias:-0.9038094072587852, loss:0.5891713791774147
Epoch:235, w1:1.1119413912951284, w2:0.9017231109469693, bias:-0.9052514217292025, loss:0.5890749016998025
Epoch:236, w1:1.1133324722296838, w2:0.9026204476267617, bias:-0.9066898021307007, loss:0.5889785916928326
Epoch:237, w1:1.1147244985668654, w2:0.9035174543559166, bias:-0.9081245943763175, loss:0.5888824479662098
Epoch:238, w1:1.1161174520888446, w2:0.9044141038365165, bias:-0.9095558435308769, loss:0.5887864693582533
Epoch:239, w1:1.117511314920813, w2:0.90

Epoch:350, w1:1.275242352806101, w2:0.9988491917486197, bias:-1.0543220311936166, loss:0.5789059791337136
Epoch:351, w1:1.2766759025529768, w2:0.9996220683061109, bias:-1.055514134993699, loss:0.578824431332171
Epoch:352, w1:1.2781095215998108, w2:1.0003935866983775, bias:-1.0567048732893287, loss:0.5787429868244877
Epoch:353, w1:1.2795432075865838, w2:1.001163746828967, bias:-1.0578942525970092, loss:0.5786616452892503
Epoch:354, w1:1.2809769581900097, w2:1.0019325486581447, bias:-1.0590822793428953, loss:0.5785804064065659
Epoch:355, w1:1.2824107711228427, w2:1.0026999922016722, bias:-1.0602689598644726, loss:0.5784992698580461
Epoch:356, w1:1.2838446441331974, w2:1.0034660775296056, bias:-1.0614543004122037, loss:0.5784182353267913
Epoch:357, w1:1.285278575003882, w2:1.0042308047651207, bias:-1.0626383071511436, loss:0.5783373024973764
Epoch:358, w1:1.2867125615517445, w2:1.0049941740833601, bias:-1.0638209861625223, loss:0.578256471055835
Epoch:359, w1:1.2881466016270327, w2:1.0057

Epoch:473, w1:1.451601240269484, w2:1.0840206109322432, bias:-1.192116549463966, loss:0.5695648328610886
Epoch:474, w1:1.4530325060470226, w2:1.0846354799317144, bias:-1.1931724720224384, loss:0.5694939473893346
Epoch:475, w1:1.454463698482929, w2:1.0852491805329387, bias:-1.194227463065025, loss:0.5694231342986609
Epoch:476, w1:1.4558948170152368, w2:1.085861714873455, bias:-1.1952815245953565, loss:0.5693523933942984
Epoch:477, w1:1.4573258610861226, w2:1.08647308509123, bias:-1.1963346586058905, loss:0.5692817244822221
Epoch:478, w1:1.458756830141841, w2:1.0870832933245593, bias:-1.1973868670780674, loss:0.5692111273691457
Epoch:479, w1:1.4601877236326624, w2:1.0876923417119715, bias:-1.1984381519824663, loss:0.5691406018625208
Epoch:480, w1:1.4616185410128115, w2:1.088300232392133, bias:-1.1994885152789556, loss:0.5690701477705317
Epoch:481, w1:1.4630492817404068, w2:1.0889069675037557, bias:-1.2005379589168428, loss:0.5689997649020941
Epoch:482, w1:1.4644799452774016, w2:1.0895125

Epoch:593, w1:1.6226968048740165, w2:1.1500533464790208, bias:-1.312669127025824, loss:0.5615262330023237
Epoch:594, w1:1.6241160603459421, w2:1.1505428684455397, bias:-1.3136253289216708, loss:0.5614628280334423
Epoch:595, w1:1.6255351933410511, w2:1.15103146555842, bias:-1.3145807879764233, loss:0.5613994766042457
Epoch:596, w1:1.6269542035462763, w2:1.1515191396662021, bias:-1.315535505471761, loss:0.561336178590273
Epoch:597, w1:1.6283730906495757, w2:1.1520058926139514, bias:-1.3164894826860287, loss:0.5612729338675149
Epoch:598, w1:1.629791854339925, w2:1.152491726243256, bias:-1.317442720894257, loss:0.5612097423124128
Epoch:599, w1:1.6312104943073107, w2:1.1529766423922263, bias:-1.3183952213681824, loss:0.5611466038018569
Epoch:600, w1:1.6326290102427217, w2:1.1534606428954939, bias:-1.3193469853762676, loss:0.561083518213184
Epoch:601, w1:1.634047401838143, w2:1.1539437295842099, bias:-1.3202980141837206, loss:0.5610204854241758
Epoch:602, w1:1.6354656687865479, w2:1.15442590

Epoch:713, w1:1.7920523614247217, w2:1.2026833479883285, bias:-1.4224451174714015, loss:0.5542683486560043
Epoch:714, w1:1.7934549264752022, w2:1.2030741229362132, bias:-1.4233205208506754, loss:0.5542105931161078
Epoch:715, w1:1.7948573373399797, w2:1.2034641706252955, bias:-1.4241953154827094, loss:0.5541528789527671
Epoch:716, w1:1.7962595937958237, w2:1.2038534925071074, bias:-1.4250695023433784, loss:0.5540952060846359
Epoch:717, w1:1.7976616956200813, w2:1.2042420900301762, bias:-1.425943082406531, loss:0.5540375744306464
Epoch:718, w1:1.799063642590675, w2:1.204629964640031, bias:-1.4268160566439951, loss:0.5539799839100088
Epoch:719, w1:1.8004654344861004, w2:1.2050171177792077, bias:-1.4276884260255853, loss:0.5539224344422116
Epoch:720, w1:1.8018670710854257, w2:1.2054035508872565, bias:-1.4285601915191066, loss:0.5538649259470183
Epoch:721, w1:1.8032685521682883, w2:1.205789265400746, bias:-1.4294313540903616, loss:0.5538074583444682
Epoch:722, w1:1.8046698775148948, w2:1.20

Epoch:834, w1:1.960583653225105, w2:1.2450808828683926, bias:-1.524214287067809, loss:0.5475590666286848
Epoch:835, w1:1.9619660944978423, w2:1.2453933825453762, bias:-1.525022577207522, loss:0.5475057990045933
Epoch:836, w1:1.9633483583429445, w2:1.245705310477097, bias:-1.5258303634136523, loss:0.5474525646601838
Epoch:837, w1:1.9647304445969638, w2:1.246016667793699, bias:-1.5266376464516367, loss:0.5473993635410768
Epoch:838, w1:1.9661123530968785, w2:1.2463274556230057, bias:-1.5274444270854173, loss:0.5473461955930664
Epoch:839, w1:1.9674940836800918, w2:1.2466376750905253, bias:-1.528250706077443, loss:0.547293060762121
Epoch:840, w1:1.9688756361844308, w2:1.2469473273194551, bias:-1.5290564841886747, loss:0.547239958994381
Epoch:841, w1:1.970257010448146, w2:1.247256413430688, bias:-1.529861762178587, loss:0.54718689023616
Epoch:842, w1:1.9716382063099098, w2:1.2475649345428161, bias:-1.530666540805173, loss:0.5471338544339418
Epoch:843, w1:1.973019223608816, w2:1.2478728917721

Epoch:928, w1:2.0897369374316095, w2:1.27210143755258, bias:-1.5980890864164008, loss:0.5426905827538243
Epoch:929, w1:2.0911020404784724, w2:1.2723648369352667, bias:-1.5988531618778536, loss:0.54264022717562
Epoch:930, w1:2.09246695247872, w2:1.2726277613137433, bias:-1.5996167992459556, loss:0.5425899004448189
Epoch:931, w1:2.0938316733052247, w2:1.2728902116208718, bias:-1.6003799991591632, loss:0.5425396025208232
Epoch:932, w1:2.095196202831209, w2:1.2731521887876265, bias:-1.6011427622547179, loss:0.5424893333631571
Epoch:933, w1:2.0965605409302444, w2:1.273413693743098, bias:-1.6019050891686484, loss:0.5424390929314655
Epoch:934, w1:2.0979246874762496, w2:1.2736747274144973, bias:-1.6026669805357734, loss:0.542388881185515
Epoch:935, w1:2.099288642343491, w2:1.27393529072716, bias:-1.6034284369897038, loss:0.542338698085192
Epoch:936, w1:2.100652405406582, w2:1.2741953846045504, bias:-1.6041894591628458, loss:0.5422885435905035
Epoch:937, w1:2.102015976540481, w2:1.2744550099682

(2.1861775674844797, 1.2896739440338827, -1.651285125475325)

In [27]:
coef,intercept

(array([[0.905744 ],
        [0.9045079]], dtype=float32),
 array([-0.09708446], dtype=float32))